# Udemy Grabber

In [1]:
import requests
from requests.utils import urlparse
from bs4 import BeautifulSoup, NavigableString
from bs4.element import Tag
import pandas as pd
from random import randint
from time import sleep
import re
from datetime import datetime, timedelta
from urllib.parse import parse_qs, unquote
import json
from Udemy_Auth import GIST_TOKEN, LOGIN, PASS
import glob
import os
from numpy import datetime64, timedelta64
import asyncio
from aiohttp import ClientSession
from Udemy_Grabber_Objects import *

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Encoding': 'gzip, deflate',
           'Upgrade-Insecure-Requests': '1',
           'Accept-Language': 'de;q=0.8',
          }

sess = requests.Session()
sess.headers.update(headers)

# Gather Courses

In [3]:
def grab(url: str, timeout=40):
    return BeautifulSoup(sess.get(url, timeout=timeout).text, 'lxml')

In [4]:
async def fetch(url, session):
    async with session.get(url) as response:
        resp = await response.text()
        return BeautifulSoup(resp, 'lxml')

In [5]:
async def run(urls):
    tasks = []
    async with ClientSession(headers=headers) as session:
        tasks = (asyncio.ensure_future(fetch(url, session)) for url in urls)
        return await asyncio.gather(*tasks)

## jucktion.com

In [6]:
urls = ['https://www.jucktion.com/forum/udemy-coupon/{}0'.format(i) for i in range(0, 7, 2)]
jucktion = await run(urls)

links = []
for soup in jucktion:
    for link in soup.find_all('td', class_='subject windowbg2'):
        l = link.find('a', href=True)['href']
        links.append(l[:l.find('?PHP')])

courses = await run(links)
jucktion_courses = set([course.find('a', class_='bbc_link')['href'] for course in courses if course.find('a', class_='bbc_link')])

In [7]:
print('Found {} courses.'.format(len(jucktion_courses)))

Found 60 courses.


## bez-postovneho.cz

In [8]:
urls = ['https://bez-postovneho.cz/?page={}'.format(i) for i in range(1, 6)]
bez = await run(urls)

links = []
for soup in bez:
    tags = soup.find_all(href=re.compile("free-udemy-course/"))
    links += [link['href'] for link in tags]

bez_courses = set()
courses = await run(set(links))
for course in courses:
    url = course.find('a', class_='btn btn-success', href=True)
    if url and url['href'].find('couponCode') > -1:
        clean = unquote(url['href'])
        bez_courses.add(clean[clean.find('https://www.udemy.com/course'):])

In [9]:
print('Found {} courses.'.format(len(bez_courses)))

Found 1 courses.


## Smartbro

In [10]:
smartbro_courses = set()
soup = grab('https://smartybro.com/category/udemy-coupon-100-off/')
links = soup.find_all('a', href=re.compile("https://smartybro.com/202"))
urls = set([link['href'][:link['href'].rfind('/')] for link in links])

courses = await run(urls)
for soup in courses:
    tag = soup.find('a', href=re.compile('udemy.com/course/[\S]+/?couponCode='))
    if tag:
        l = tag['href']
        smartbro_courses.add(l[:l.find('&')])

In [11]:
print('Found {} courses.'.format(len(smartbro_courses)))

Found 15 courses.


## couponscorpion

In [12]:
urls = ['https://couponscorpion.com/page/{}/'.format(i) for i in range(1, 10)]
scorpion = await run(urls)

links = []
for soup in scorpion:
    for article in soup.find_all('div', 'info_in_dealgrid'):
        if article.find('span', 'grid_onsale').text == '100% OFF':
            links.append(article.find('a', href=True, class_=False)['href'])

scorpion_courses = set()
regex = re.compile("[\w]{20,}%3D&s=[\w]+&n=[\w]+")
courses = await run(links)
for soup in courses:
    for script in soup.find_all('script', {'type':False, 'data-cfasync':False, 'src':False}):
        token = re.findall(regex, script.text)
        if token:
            try:
                url = sess.get('https://couponscorpion.com/scripts/udemy/out.php?go=' + token[0]).url
                coupon = parse_qs(urlparse(url).query)['couponCode']
                scorpion_courses.add(url[:url.find('?')] + '?couponCode=' + str(coupon[0]))
            except:
                print('error getting', url, '\nTrying again.')
                url = sess.get('https://couponscorpion.com/scripts/udemy/out.php?go=' + token[0]).url
                coupon = parse_qs(urlparse(url).query)['couponCode']
                scorpion_courses.add(url[:url.find('?')] + '?couponCode=' + str(coupon[0]))

In [13]:
print('Found {} courses.'.format(len(scorpion_courses)))

Found 28 courses.


## discudemy

In [14]:
urls = ['https://www.discudemy.com/all/{}'.format(i) for i in range(1, 6)]
discudemy = await run(urls)

links = []
for soup in discudemy:
    for div in soup.find_all('div', 'content'):
        if div.find('div', 'header')and div.find('div', 'meta').text.find('->') > 0:
            links.append(div.find('a', href=True)['href'])

regex = re.compile("/[\w\d\-]+/")
courses = await run([regex.sub('/go/', link) for link in links])
discudemy_courses = set(soup.find('a', href=True, class_=False, target='_blank')['href'] for soup in courses)

In [15]:
print('Found {} courses.'.format(len(discudemy_courses)))

Found 69 courses.


## oodemy

In [16]:
soup = grab('https://www.schoobly.com/search?updated-max=2025-02-28T21:09:00%2B05:30&max-results=30&reverse-paginate=true')
oodemy = set([tag['href'] for tag in soup.find_all('a', 'read-more', href=True)])
soup = grab(soup.find('a', 'blog-pager-older-link')['href'])
oodemy = oodemy.union([tag['href'] for tag in soup.find_all('a', 'read-more', href=True)])

oodemy_courses = set()
courses = await run(oodemy)
for soup in courses:
    link = soup.find('a', 'read-more', target=True, href=True)
    if link and link['href'].find('couponCode') > -1:
        oodemy_courses.add(link['href'])

In [17]:
print('Found {} courses.'.format(len(oodemy_courses)))

Found 32 courses.


## freebiesglobal
We have to split the async requests.

In [18]:
urls = ['https://freebiesglobal.com/dealstore/udemy/page/{}'.format(i) for i in range(1, 5)]
freebies = [grab(url) for url in urls]
links = [span.next.next.next['href'] for soup in freebies for span in soup.find_all('span', 'grid_onsale')]

freebies_courses = []
counter = [int(len(links)/2), len(links)]
for c, i in enumerate(counter):
    courses = await run(links[max(0, i-counter[c-1]-1):i])
    freebies_courses += [course.find('a', 're_track_btn', href=True)['href'] for course in courses if course.find('a', 're_track_btn', href=True)]
freebies_courses = set(freebies_courses)

In [19]:
print('Found {} courses.'.format(len(freebies_courses)))

Found 94 courses.


## Google

In [20]:
url = 'https://www.google.com/search?q=allinurl:+couponcode+site:udemy.com/course&lr=&safe=strict&hl=de&tbs=qdr:w&filter=0&biw=1920&bih=966'
soup = grab(url)
google = [unquote(a['href'][7:a['href'].find('&sa')]) for a in soup.find_all('a', href=True) if a['href'].find('udemy.com/course/') > -1]
soup = grab(url + '&start=10')
google += [unquote(a['href'][7:a['href'].find('&sa')]) for a in soup.find_all('a', href=True) if a['href'].find('udemy.com/course/') > -1]

In [21]:
print('Found {} courses.'.format(len(set(google))))

Found 16 courses.


## Merge Courses

In [22]:
udemy = scorpion_courses | Reddit().courses | bez_courses | Freesamples().courses \
        | discudemy_courses | Facebook().courses | smartbro_courses | Dsmenders().courses | \
        jucktion_courses | oodemy_courses | freebies_courses | set(google)

## Analyse
Find courses on Udemy, verify that they are free but not permanently, and crawl metadata.

In [23]:
list_of_files = glob.glob('./udemy-courses/*.csv')
file_day_before = list_of_files[-1]
file_old_courses = list_of_files[-2]

In [24]:
day_before = pd.read_csv(file_day_before)
old_courses = pd.read_csv(file_old_courses)

In [25]:
parser = UdemyParser(udemy)
await parser.parse()

In [26]:
df = parser.df

## Find valuable Courses with high rating

In [27]:
min_num_reviews = 9
min_avg_rating = 3.0

In [28]:
instructor_block = set(['Srinidhi Ranganathan', 'Learn Tech Plus', 'ABD Muhammad',
                        'Bardha I, MBA', 'Pronoy Kumar', 'Mohammad Mostafizur Rahaman',
                        'Shokal Shah', 'Masud Rana', 'Muhammad Saju Ahmed', 'Marufa Yasmin',])
mask = [not bool(instructor_block & set(instr)) for instr in list(df.instructor)]

In [29]:
valuable = df[((df.language=='Deutsch') | ((df.rating > min_avg_rating) & \
              (df.num_ratings > min_num_reviews))) & mask].sort_values(by=['language', 'topic_0', 'topic_1'])

In [30]:
valuable

,title,link,rating,num_ratings,language,duration,topic_0,topic_1,topic_2,expiring,instructor
11,Kundengewinnung und Kundenbindung mit Wowing,https://www.udemy.com/course/kundengewinnung-u...,0.00000,0,Deutsch,2 total Stunden,Marketing,Branding,NaN,2 days 11:00:00,{Sebastian Glöckner}
86,Agile Project Management: Scrum Step by Step w...,https://www.udemy.com/course/scrum-master-trai...,4.20208,2746,English,2 total Stunden,Business,Management,NaN,0 days 00:00:00,{Paul Ashun}
18,AMAZON MASTERY - How To Become An Amazon Best ...,https://www.udemy.com/course/amazon-best-selli...,3.65000,43,English,"2,5 total Stunden",Business,Medien,NaN,1 days 14:00:00,"{Ricky Shetty, Scott Paton, Scott Alex}"
28,The HR professional’s guide to cybersecurity,https://www.udemy.com/course/the-hr-profession...,3.17857,15,English,1 total Stunde,Business,Personalwesen,NaN,2 days 04:00:00,{Teo Dvali}
96,Dropshipping with Wordpress: Create a Dropship...,https://www.udemy.com/course/dropshipping-with...,3.85000,62,English,"2,5 total Stunden",Business,Selbstständigkeit,NaN,0 days 13:00:00,"{MindQuest Academy, Scott Paton, Scott Alex}"
...,...,...,...,...,...,...,...,...,...,...,...
83,C# ile Web Kazıma (Web Scraping & Web Crawling...,https://www.udemy.com/course/c-ile-web-kazma-w...,3.95000,13,Türkçe,3 total Stunden,IT & Software,Sonstiges,NaN,1 days 02:00:00,{Halil AKCIL}
61,Golang Programlama Dili : Standart Kütüphane,https://www.udemy.com/course/golang-programlam...,3.90000,35,Türkçe,"10,5 total Stunden",Programmierung,Programmiersprachen,NaN,0 days 04:00:00,{Cihan Özhan}
70,Golang Programlama Dili,https://www.udemy.com/course/go-programlama-di...,4.18750,210,Türkçe,17 total Stunden,Programmierung,Softwareentwicklung,NaN,0 days 04:00:00,{Cihan Özhan}
136,Golang ile Web API Programlama,https://www.udemy.com/course/golang-restful-ap...,4.50000,60,Türkçe,18 total Stunden,Programmierung,Webentwicklung,NaN,0 days 04:00:00,{Cihan Özhan}


In [31]:
path = datetime.now().strftime('%Y-%m-%d %H-%M')
valuable.to_csv('./udemy-courses/' + path + '.csv')

In [32]:
yesterdays_courses = [course.title for course in day_before.itertuples() if not course.title in list(old_courses.title)]
new_courses = [course.title for course in valuable.itertuples() if not course.title in list(day_before.title)]

In [33]:
df[df.language == 'Deutsch']

,title,link,rating,num_ratings,language,duration,topic_0,topic_1,topic_2,expiring,instructor
11,Kundengewinnung und Kundenbindung mit Wowing,https://www.udemy.com/course/kundengewinnung-u...,0.0,0,Deutsch,2 total Stunden,Marketing,Branding,NaN,2 days 11:00:00,{Sebastian Glöckner}


In [34]:
print('{} kostenlose Kurse.'.format(len(df)))
print('{} wertvolle Kurse.'.format(len(valuable)))
print('{} neue Kurse'.format(len(new_courses)))

150 kostenlose Kurse.
77 wertvolle Kurse.
47 neue Kurse


In [35]:
languages = valuable.language.unique()

In [36]:
print(languages)

['Deutsch' 'English' 'Español' 'Türkçe' 'हिन्दी']


### Edit Gist

In [37]:
GIST="https://api.github.com/gists/bc90582aadd7d466fef67aed524d886d"

In [38]:
git_headers={'Authorization':'token %s'%GIST_TOKEN}
params={'scope':'gist'}

In [39]:
content = '\n'.join(s for s in valuable.link.values)

In [40]:
payload={"description":"Kostenlose und gut bewertete Kurse",
         "public":True,
         "files":{
             "gistfile1.txt":{
                 "content":content}}}

In [41]:
res = requests.patch(GIST,
                     headers=git_headers,
                     params=params,
                     data=json.dumps(payload))
if res.ok:
    print('Successfully patched gist.')

Successfully patched gist.


## Edit Deal
* login to mydealz
* manually open the deal and delete the old list
* run script to create a new list

In [42]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys

In [43]:
options = Options()
#options.headless = True
driver = webdriver.Firefox(options=options)

### Login to mydealz

In [44]:
driver.get('https://www.mydealz.de/login')
driver.find_element_by_id('login_form-identity').send_keys(LOGIN)
driver.find_element_by_id('login_form-password').send_keys(PASS)
driver.find_element_by_id('login_form-password').send_keys(Keys.ENTER)
sleep(5)

In [45]:
driver.get('https://www.mydealz.de/deals/udemy-kostenlose-und-gut-bewertete-kurse-1533722/edit')

## Manual step: delete old list and adjust description

In [46]:
bold = driver.find_element_by_xpath("//button[@class='toolbar-btn toolbar-btn--bold space--mr-1']/span")
li = driver.find_element_by_xpath("//button[@class='toolbar-btn toolbar-btn--unorderedlist space--mr-1']/span")
link = driver.find_element_by_xpath("//button[@class='cept-linkBtn space--mr-1']/span")
editor = driver.find_element_by_xpath("//div[@class='redactor-box aGrid']/div")
editor.click()
editor.send_keys(Keys.CONTROL, Keys.END)

In [47]:
lang = {'Deutsch': 'Deutsch (ohne Qualitätsfilter)',
        'English': 'Englisch',
        'Español': 'Spanisch',
        'العربية': 'Arabisch',
        'Italiano': 'Italienisch',
        'Türkçe': 'Türkisch',
        'English (India)': 'Englisch (Indisch)',
        'Русский': 'Russisch',
        'Français': 'Französisch',
        '中文': 'Chinesisch',
        '日本語': 'Japanisch',
        'Português': 'Portugiesisch',
        '한국어': 'Koreanisch',
        'हिन्दी': 'Hindi'}

In [48]:
def list_courses(courses):
    li.click()
    for course in courses.itertuples():
        if course.expiring < timedelta64(24,'h'):
            editor.send_keys('⏳ ')
        if course.title in yesterdays_courses:
            editor.send_keys('*gestern ')
        elif course.title in new_courses:
            editor.send_keys('*neu ')
        if course.rating > 4.5:
            bold.click()
        link.click()
        driver.find_element_by_id('linkPopover-href').send_keys(course.link)
        driver.find_element_by_id('linkPopover-content').send_keys(course.title)
        driver.find_element_by_id('linkPopover-content').send_keys(Keys.RETURN)
        if course.rating > 4.5:
            bold.click()
        rounded = float(round(course.rating * 2, 0)/2)
        stars = '★' * int(rounded) + '✯' * bool(rounded % 1)
        stars += '☆' * (5 - len(stars))
        num_ratings = '({} Bewertungen)'.format(int(course.num_ratings))
        length = course.duration.replace('total ', '').replace(' gesamt', '')
        editor.send_keys(', ' + stars + ' ' + num_ratings + ', ' + length)

        editor.send_keys(Keys.RETURN)
        sleep(0.5)
    editor.send_keys(Keys.RETURN)

In [49]:
for language in languages:
    bold.click()
    editor.send_keys(lang[language])
    bold.click()
    editor.send_keys(Keys.ENTER)

    val_lang = valuable[valuable.language == language]
    main_topics = val_lang.topic_0.unique()
    for main_topic in main_topics:
        editor.send_keys(main_topic)
        editor.send_keys(Keys.RETURN)
        list_courses(val_lang[valuable.topic_0 == main_topic])
    editor.send_keys(Keys.RETURN)

D:\Programme\Anaconda\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


## Subscribe to Courses
Login manually

In [ ]:
driver.quit()

In [ ]:
driver = webdriver.Firefox(options=options)

In [ ]:
driver.get('https://www.udemy.com')

In [50]:
new_links = [course.link for course in valuable.itertuples() if not course.title in list(day_before.title)]
error_links = []
for link in new_links:
    driver.get(link)
    try:
        sleep(randint(4, 10))
        btn = driver.find_element_by_xpath("//button[@data-purpose='buy-this-course-button']")
        if btn.text == 'Jetzt einschreiben':
            btn.click()
            sleep(6)
            if driver.current_url.find('success') < 0:
                error_links.append(link)
        else:
            print(link, 'already subscribed.')
    except:
        print(link, '# already subscribed.')

https://www.udemy.com/course/windows-server-2012-r2-installation-and-configuration/?couponCode=67016E1E0247CD20D53E already subscribed.
https://www.udemy.com/course/server-2016-hands-on-training/?couponCode=11AD3E28087B73CC0D9B already subscribed.
https://www.udemy.com/course/microsoft-windows-server-2019-hands-on-training-part-i/?couponCode=2AC2D131259E473D3BC5 already subscribed.
https://www.udemy.com/course/linux-made-easy-with-hands-on-training/?couponCode=4B1CEB85E07DECA880EC already subscribed.
https://www.udemy.com/course/windows-server-2012r2-hands-on-training-part-ii/?couponCode=B8B5DDF7194931BF1E4F already subscribed.
https://www.udemy.com/course/hacking-made-easy-with-hands-on-training/?couponCode=1AC6A6C7FAE3F5092315 already subscribed.
https://www.udemy.com/course/ethical-hacking-capture-the-flag-walkthroughs/?couponCode=910C7FB0458866FECC31 already subscribed.
https://www.udemy.com/course/ethical-hacking-capture-the-flag-walkthroughs-v2/?couponCode=5048CD2346D8BFFEC714 al

In [51]:
print('Failed to subscribe to {} courses.'.format(len(error_links)))

Failed to subscribe to 16 courses.


In [52]:
for link in error_links:
    driver.get(link)
    try:
        sleep(randint(4, 10))
        btn = driver.find_element_by_xpath("//button[@data-purpose='buy-this-course-button']")
        if btn.text == 'Jetzt einschreiben':
            btn.click()
            sleep(6)
            if driver.current_url.find('success') < 0:
                print('error subscribing to', link)
        else:
            print(link, 'already subscribed.')
    except:
        print(link, '# already subscribed.')

error subscribing to https://www.udemy.com/course/12-proven-ways-to-turn-youtube-into-a-career/?couponCode=F8E3772C8BE15A356259
error subscribing to https://www.udemy.com/course/entrepreneurship-accelerator-program/?couponCode=BBF6A4AB54B50A47DC6B
error subscribing to https://www.udemy.com/course/deploy-spring-boot-to-azure/?couponCode=FOCUS-AND-LEARN
error subscribing to https://www.udemy.com/course/deploy-java-spring-boot-to-aws-amazon-web-service/?couponCode=FOCUS-AND-LEARN
error subscribing to https://www.udemy.com/course/learn-pivotal-cloud-foundry-pcf-deploying-spring-boot-apps/?couponCode=FOCUS-AND-LEARN
error subscribing to https://www.udemy.com/course/devops-with-docker-kubernetes-and-azure-devops/?couponCode=FOCUS-AND-LEARN
error subscribing to https://www.udemy.com/course/fatherhood/?couponCode=MARCH27FATHERHOOD
error subscribing to https://www.udemy.com/course/full-stack-application-with-spring-boot-and-react/?couponCode=FOCUS-AND-LEARN
error subscribing to https://www.udem

In [113]:
driver.quit()

In [ ]:
print(this_is_an_error)

In [ ]:
base = 'https://www.mydealz.de/?page={}'

In [ ]:
deals = []
for i in range(1, 500):
    soup = BeautifulSoup(sess.get(base.format(i)).text, 'lxml')
    links = soup.find_all('a', 'cept-thread-image-link imgFrame imgFrame--noBorder thread-listImgCell', href=True)
    deals += [link['href'] for link in links]

In [ ]:
df = pd.DataFrame(columns=['title', 'published', 'time', 'hour', 'weekday', 'temperature', 'url'])
for deal in set(deals):
    soup = BeautifulSoup(sess.get(deal).text, 'lxml')
    if soup.find('span', 'cept-vote-temp vote-temp vote-temp--burn'):
        published = datetime.fromisoformat(soup.find('meta', {'property':'og:article:published_time'})['content'])
        title = soup.find('meta', {'property':'og:title'})['content'][:-13]
        temperature = int(soup.find('span', 'cept-vote-temp vote-temp vote-temp--burn').text.strip()[:-1])
        time = published.time()
        hour = time.hour
        weekday = int(published.weekday())

        data = {'title':title,
                'published':published,
                'time':time,
                'hour':hour,
                'weekday':weekday,
                'temperature':temperature,
                'url':deal}
        df = df.append(data, ignore_index=True)

In [ ]:
df.weekday = pd.to_numeric(df.weekday)
df.temperature = pd.to_numeric(df.temperature)

In [ ]:
df.groupby(['hour', 'weekday']).median().sort_values('temperature')

In [ ]:
labels = ['nachts', 'morgens', 'nachmittags', 'abends']
pd.cut(df.hour, range(0, 24, 6), right=False, labels=labels)

In [59]:
driver = webdriver.Firefox()

In [60]:
driver.get('https://ibmcoders.ibmviprewards.ibm.com/channels/266')

In [63]:
for _ in range(300):
    try:
        driver.get('https://ibmcoders.ibmviprewards.ibm.com/channels/266')
        driver.find_element_by_xpath("//div[@id='challenge-862']/div/div/div/div/a").click()
        sleep(1.5)
        driver.find_element_by_xpath('//section[@class="pt-modal large available"]').click()
        sleep(3)
        driver.find_element_by_xpath('//section[@class="pt-modal large available"]').send_keys(Keys.DOWN)
        driver.find_element_by_xpath('//section[@class="pt-modal large available"]').send_keys(Keys.DOWN)
        driver.find_element_by_xpath('//section[@class="pt-modal large available"]').send_keys(Keys.DOWN)
        driver.find_element_by_xpath('//section[@class="pt-modal large available"]').send_keys(Keys.DOWN)
        driver.find_element_by_xpath('//section[@class="pt-modal large available"]').send_keys(Keys.DOWN)
        driver.find_element_by_xpath('//section[@class="pt-modal large available"]').send_keys(Keys.DOWN)
        driver.find_element_by_xpath('//section[@class="pt-modal large available"]').send_keys(Keys.DOWN)
        driver.find_element_by_xpath("//div[@class='pt-form-actions']/div/div/input").click()
        sleep(0.5)
        driver.find_element_by_xpath("//div[@class='dialog-header']/button").click()
    except:
        print('oops...')

oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...
oops...


In [ ]:
driver.get('https://ibmcoders.ibmviprewards.ibm.com/channels/266')
driver.find_element_by_xpath("//div[@id='challenge-862']/div/div/div/div/a").click()
sleep(1.5)
driver.find_element_by_xpath('//section[@class="pt-modal large available"]').click()
sleep(3)

In [67]:
driver.quit()